In [34]:
!pip install pandas scikit-learn nltk


In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import nltk
import requests
import zipfile
import os


In [36]:
# Download the ZIP file
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip"
response = requests.get(url)

zip_file_path = "smsspamcollection.zip"
with open(zip_file_path, 'wb') as f:
    f.write(response.content)

# Extract the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall("smsspamcollection")


In [37]:
# Load the dataset from the extracted file
df = pd.read_csv("smsspamcollection/SMSSpamCollection", sep='\t', names=['label', 'message'])


In [38]:
#Check for missing values
print("Missing values in the dataset:")
print(df.isnull().sum())

Missing values in the dataset:
label      0
message    0
dtype: int64


In [39]:
#Handle missing values
df.dropna(inplace=True)


In [40]:
# Convert labels to binary values
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

# Display the first few rows
print(df.head())


   label                                            message
0      0  Go until jurong point, crazy.. Available only ...
1      0                      Ok lar... Joking wif u oni...
2      1  Free entry in 2 a wkly comp to win FA Cup fina...
3      0  U dun say so early hor... U c already then say...
4      0  Nah I don't think he goes to usf, he lives aro...


In [41]:
#Check for missing values again
print("Missing values in the dataset after handling:")
print(df.isnull().sum())

Missing values in the dataset after handling:
label      0
message    0
dtype: int64


In [42]:
#Split the Data
X_train, X_test, y_train, y_test = train_test_split(df['message'], df['label'], test_size=0.2, random_state=42)

# Check for NaN values in y_train
print("Missing values in y_train:")
print(y_train.isnull().sum())


Missing values in y_train:
0


In [43]:
#Text Vectorization
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)


In [44]:
#Train the Model
model = MultinomialNB()
model.fit(X_train_vectorized, y_train)


MultinomialNB()

In [45]:
#Make Predictions
y_pred = model.predict(X_test_vectorized)

In [46]:
#Evaluate the Model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print(report)

Accuracy: 0.99
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       966
           1       1.00      0.94      0.97       149

    accuracy                           0.99      1115
   macro avg       1.00      0.97      0.98      1115
weighted avg       0.99      0.99      0.99      1115



In [60]:
#Function to predict if a new message is spam or not
def predict_message(message):
    # Transform the message using the same vectorizer
    message_vectorized = vectorizer.transform([message])

    # Predict using the trained model
    prediction = model.predict(message_vectorized)

    # Convert prediction to label
    if prediction[0] == 1:
        return "Spam"
    else:
        return "Not Spam"

#Input a new message
new_message = input("Enter a message to check if it's spam or not: ")

#Predict the label of the new message
result = predict_message(new_message)
print(f"The message is: {result}")


Enter a message to check if it's spam or not: CONGRATS UR NUMBER IS SELECTED AS Winner, OF 875,000 BRITISH POUND FROM U.N HUMANITARIAN AID PLEASE EMAIL US UR NUMBER,AGE,NAME,ADDRESS:unteampro6@hotmail.com
The message is: Spam
